In [1]:
import numpy as np

import pickle
from feeder import tools

import torch
import torch.nn as nn

from net import st_gcn

In [2]:
class Feeder(torch.utils.data.Dataset):
    """ Feeder for skeleton-based action recognition
    Arguments:
        data_path: the path to '.npy' data, the shape of data should be (N, C, T, V, M)
        label_path: the path to label
        random_choose: If true, randomly choose a portion of the input sequence
        random_shift: If true, randomly pad zeros at the begining or end of sequence
        window_size: The length of the output sequence
        normalization: If true, normalize input sequence
        debug: If true, only use the first 100 samples
    """

    def __init__(self,
                 data_path,
                 label_path,
                 random_choose=False,
                 random_move=False,
                 window_size=-1,
                 debug=False,
                 mmap=True):
        self.debug = debug
        self.data_path = data_path
        self.label_path = label_path
        self.random_choose = random_choose
        self.random_move = random_move
        self.window_size = window_size

        self.load_data(mmap)

    def load_data(self, mmap):
        # data: N C V T M

        # load label
        with open(self.label_path, 'rb') as f:
            self.sample_name, self.label = pickle.load(f)

        # load data
        if mmap:
            self.data = np.load(self.data_path, mmap_mode='r')
        else:
            self.data = np.load(self.data_path)

        if self.debug:
            self.label = self.label[0:100]
            self.data = self.data[0:100]
            self.sample_name = self.sample_name[0:100]

        self.N, self.C, self.T, self.V, self.M = self.data.shape

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        # get data
        data_numpy = np.array(self.data[index])
        label = self.label[index]

        # processing
        if self.random_choose:
            data_numpy = tools.random_choose(data_numpy, self.window_size)
        elif self.window_size > 0:
            data_numpy = tools.auto_pading(data_numpy, self.window_size)
        if self.random_move:
            data_numpy = tools.random_move(data_numpy)

        return data_numpy, label

In [3]:
label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]

In [4]:
dev = "cuda:0"

label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]

data_path = "./data/kinetics-sharing/train_data.npy"
label_path = "./data/kinetics-sharing/train_label.pkl"
dset = Feeder(data_path, label_path)

data_loader = torch.utils.data.DataLoader(
    dataset=dset,
    batch_size=64,
    shuffle=False)

graph_args = {'layout': 'openpose', 'strategy': 'spatial'}

model = st_gcn.Model(in_channels=3, num_class=400,
                     graph_args=graph_args, edge_importance_weighting=True)
weights = torch.load("./models/st_gcn.kinetics.pt")
model.load_state_dict(weights)
model.eval()

model = model.to(dev)

loss_value = []
result_frag = []
label_frag = []

In [5]:
output_all = []
voting_label_all = []
label_all = []

In [6]:
for data, label in data_loader:
    data = data.float().to(dev)
    label = label.long().to(dev)

    with torch.no_grad():
        output_batch, feature_batch = model.extract_feature(data)     

        for i in range (len(label)):
            output = output_batch[i]
            feature = feature_batch[i]
            intensity = (feature * feature).sum(dim=0) ** 0.5
            intensity = intensity.cpu().detach().numpy()

            # classification result of the full sequence
            voting_label = output.sum(dim=3).sum(
                dim=2).sum(dim=1).argmax(dim=0)
            voting_label_name = label_name[voting_label]
            print("voting_label_name is ", voting_label_name)

            real_label_name = label_name[label[i]]
            print("real label is", real_label_name)

            output_all.append(output.data.cpu().numpy())
            voting_label_all.append(voting_label.data.cpu().numpy())
            label_all.append(label[i].data.cpu().numpy())

voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  using computer
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is 

real label is shaking hands
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowki

real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is shaking hands
voting_label_n

voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is clapping
voting_label_name 

real label is clapping
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  finger snapping
real label is counting money
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowk

...

voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_

voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_na

voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is clapping
voting_labe

voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is clapping
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is hugging
voti

voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is counting money
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is writing
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is shaking hands
voting_label_name is  snowkiting
real label is hugging
voting_label_name is  snowkiting
real label is cutting nails
voting_label_name is  snowkiting
real label is cutti

In [7]:
np.shape(output_all)

(2635, 400, 75, 18, 2)

In [8]:
np.shape(label_all)

(2635,)

In [9]:
def resize(x):
    N, C, T, V, M = x.size()
    x = x.permute(0, 4, 3, 1, 2).contiguous()
    x = x.view(N * M, V * C * T)
    return x

In [ ]:
tsne_data = resize(torch.tensor(output_all))
tsne_data.shape

In [ ]:
data = tsne_data[::2,]
label = np.array(label_all)
print(data.shape)
print(label.shape)

In [ ]:
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.manifold import TSNE

def map_label(lable_num):
    if lable_num == 57:
        color = 1
    elif lable_num == 396:
        color = 2
    elif lable_num == 158:
        color = 3
    elif lable_num == 288:
        color = 4
    elif lable_num == 74:
        color = 5
    elif lable_num == 81:
        color = 6
    return color

def plot_embedding_2D(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)
    fig = plt.figure()
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(map_label(label[i])),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    return fig
def plot_embedding_3D(data,label,title): 
    x_min, x_max = np.min(data,axis=0), np.max(data,axis=0) 
    data = (data- x_min) / (x_max - x_min) 
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d') 
    for i in range(data.shape[0]): 
        ax.text(data[i, 0], data[i, 1], data[i,2],str(label[i]), color=plt.cm.Set1(map_label(label[i])),fontdict={'weight': 'bold', 'size': 9}) 
    return fig

In [ ]:
print('Begining......') #时间会较长，所有处理完毕后给出finished提示
tsne_2D = TSNE(n_components=2, init='pca', random_state=0) #调用TSNE
result_2D = tsne_2D.fit_transform(data)
# tsne_3D = TSNE(n_components=3, init='pca', random_state=0)
# result_3D = tsne_3D.fit_transform(data)
print('Finished......')

In [ ]:
#调用上面的两个函数进行可视化
fig1 = plot_embedding_2D(result_2D, label,'t-SNE')
plt.show(fig1)
# fig2 = plot_embedding_3D(result_3D, label,'t-SNE')
# plt.show(fig2)

In [ ]:
np.unique(label)